# Requirement

In [1]:
! pip install numpy
! pip install scikit-learn
! pip install torch
!pip install datasets transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import torch

import os
from datasets import load_dataset
torch.cuda.is_available()

False

# process text data

Upload your data via the document button in the left side.

In [36]:
from google.colab import drive
drive.mount('/content/drive')
dataset = load_dataset('csv', data_files = os.path.join('data', 'evp.train.csv'))
print(dataset['train'][0]) # The dataset should contain two key: "text"(data) and "level"(label, the corresponding class)
dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
{'text': 'My mother is having her car repaired.', 'level': 'B1'}


DatasetDict({
    train: Dataset({
        features: ['text', 'level'],
        num_rows: 20720
    })
})

In [5]:
from transformers import AutoTokenizer

MODEL_NAME = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [6]:
# Tokenize one sample
example = dataset['train']['text'][0]

embeddings = tokenizer(example)
embeddings

{'input_ids': [101, 2026, 2388, 2003, 2383, 2014, 2482, 13671, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

# process label

In [7]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse = False)
encoder = encoder.fit(np.reshape(dataset['train']['level'], (-1, 1)))
LABEL_COUNT = len(encoder.categories_[0])
print(LABEL_COUNT)

6


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [8]:
embeddings["label"]=encoder.transform([[dataset['train']['level'][0]]])
embeddings

{'input_ids': [101, 2026, 2388, 2003, 2383, 2014, 2482, 13671, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'label': array([[0., 0., 1., 0., 0., 0.]])}

In [9]:
embeddings["label"]=encoder.transform([[x] for x in dataset['train']['level']])
embeddings

{'input_ids': [101, 2026, 2388, 2003, 2383, 2014, 2482, 13671, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'label': array([[0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])}

# preprocess dataset

In [26]:
def preprocess(dataslice):

    embeddings=tokenizer(dataslice["text"], padding=True, truncation=True)
    embeddings["label"]=encoder.transform([[x] for x in dataslice['level']])
    del dataslice['text']
    del dataslice['level']

    return embeddings

In [30]:
processed_data = dataset.map(preprocess,batched = True)
processed_data["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'label'],
    num_rows: 20720
})

# start training

In [31]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,
                                                           num_labels = LABEL_COUNT)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier

In [32]:
train_val_dataset = processed_data['train'].train_test_split(test_size = 0.3)
print(train_val_dataset)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'label'],
        num_rows: 14504
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'label'],
        num_rows: 6216
    })
})


In [33]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [34]:
from transformers import TrainingArguments, Trainer

LEARNING_RATE = 0.005
BATCH_SIZE = 32
EPOCH = 2
training_args = TrainingArguments(
    output_dir = 'model',
    learning_rate = LEARNING_RATE,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    num_train_epochs = EPOCH,
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_val_dataset["train"],
    eval_dataset = train_val_dataset["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
)

In [35]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


KeyboardInterrupt: ignored

In [ ]:
model.save_pretrained(os.path.join('model', 'finetuned'))

In [ ]:
from transformers import AutoModelForSequenceClassification

mymodel = AutoModelForSequenceClassification.from_pretrained(os.path.join('model', 'finetuned'))

In [ ]:
examples = [
    # A2
    "Remember to write me a letter.",
    # B2
    "Strawberries and cream - a perfect combination.",
    "This so-called \"Perfect Evening\" was so disappointing, as well as discouraging us from coming to your Circle Theatre again.",
    # C1
    "Some may altogether give up their studies, which I think is a disastrous move.",
]

In [ ]:
input = tokenizer(examples, truncation=True, padding=True, return_tensors="pt")
logits = mymodel(**input).logits
logits

In [ ]:
from torch import nn

predicts = nn.functional.softmax(logits, dim = -1)
predicts